## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:

- [Getting Started](#Getting-Started)
    - [initialization](#initialization)
    - [show available hardware info list](#show-available-hardware-information)
    - [choose hardware config](#Set-default-hardware-config-Optional)
- [Creating And Deploying Task](#Creating-And-Deploying-Task)
    - [estimate payment amount](#estimate-payment-amount-optional)
    - [Auto Pay Path](#Auto-Pay-Path)
        - [create task with auto_pay](#Create-task-auto_pay)
        - [renew task with auto-pay](#renew-task-auto-pay-optional)
    - [No Auto Pay Path](#No-Auto-Pay-Path)
        - [create task with no auto_pay to get `task_uuid`](#Create-task-No-auto_pay)
        - [make payment](#make-payment-Optional)
        - [validate payment](#validate-payment-to-deploy-task)
        - [renew task](#renew-task-no-auto-pay-optional)
    - [terminate task](#terminate-task-optional)
    - [claim review](#claim-review-optional)
- [View Deployed Task Results](#View-Deployed-Task-Results)
    - [follow up task status](#follow-up-task-status-optional)
    - [show result](#show-result)


# Getting Started


### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK demo, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
from swan import Orchestrator
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

wallet_address = os.getenv('WALLET')
private_key = os.getenv('PK')
api_key = os.getenv('API_KEY')
# for testnet dev
swan_orchestrator: Orchestrator = swan.resource(
    api_key=api_key, 
    service_name='Orchestrator', 
    login_url='https://swanhub-cali.swanchain.io',      # dev version for testnet login url
    url_endpoint='https://swanhub-cali.swanchain.io'    # dev version for testnet
)

# # for testnet prod
# swan_orchestrator = swan.resource(
#     api_key=api_key, 
#     service_name='Orchestrator'
# )

# # for mainnet
# swan_orchestrator = swan.resource(
#     api_key=api_key, 
#     service_name='Orchestrator',
#     network='mainnet'
# )


ModuleNotFoundError: No module named 'cryptography'

In [2]:
r = swan_orchestrator.contract_info
print(json.dumps(r, indent=2))

{
  "client_contract_address": "0x1AcF156C8bfaC190b6004586F041959A42903790",
  "payment_contract_address": "0xB48c5D1c025655BA79Ac4E10C0F19523dB97c816",
  "rpc_url": "https://rpc-atom-internal.swanchain.io",
  "swan_token_contract_address": "0x91B25A65b295F0405552A4bbB77879ab5e38166c"
}


### Show available hardware information (Optional)

In [3]:
hardwares_info = swan_orchestrator.get_hardware_config()
print(hardwares_info)
hardware_id = 0
region = 'global'

[{'id': 0, 'name': 'C1ae.small', 'description': 'CPU only · 2 vCPU · 2 GiB', 'type': 'CPU', 'region': ['North Carolina-US', 'Quebec-CA'], 'price': '0.0', 'status': 'available'}, {'id': 1, 'name': 'C1ae.medium', 'description': 'CPU only · 4 vCPU · 4 GiB', 'type': 'CPU', 'region': ['North Carolina-US', 'Quebec-CA'], 'price': '1.0', 'status': 'available'}, {'id': 12, 'name': 'G1ae.small', 'description': 'Nvidia 3080 · 4 vCPU · 8 GiB', 'type': 'GPU', 'region': ['North Carolina-US', 'Quebec-CA'], 'price': '10.0', 'status': 'available'}, {'id': 13, 'name': 'G1ae.medium', 'description': 'Nvidia 3080 · 8 vCPU · 16 GiB', 'type': 'GPU', 'region': ['North Carolina-US', 'Quebec-CA'], 'price': '11.0', 'status': 'available'}, {'id': 20, 'name': 'Hpc1ae.small', 'description': 'Nvidia 3090 · 4 vCPU · 8 GiB', 'type': 'GPU', 'region': ['Quebec-CA'], 'price': '14.0', 'status': 'available'}, {'id': 21, 'name': 'Hpc1ae.medium', 'description': 'Nvidia 3090 · 8 vCPU · 16 GiB', 'type': 'GPU', 'region': ['Queb

### Select hardware and region (Optional)

choose a hardware with its hardware id and region. If no hardware_id is provided in future functions, it will default to free tier, and it no region is provided, it will default to global.

In [4]:
hardware_id = 0
region = 'global'

# Creating And Deploying Task

### Show repository image of pre-defined applications (optional)

In [5]:
swan_orchestrator.get_app_repo_image() 

{'data': [{'name': 'Yearn',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/Yearn'},
  {'name': 'yfii',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/yfii'},
  {'name': 'uniswap',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/uniswap'},
  {'name': 'luaswap',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/luaswap'},
  {'name': 'kanboard',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/kanboard'},
  {'name': 'swanchain-docker',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/swanchain-docker'},
  {'name': 'DEGO',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/DEGO'},
  {'name': 'pacman',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/pacman'},
  {'name': 'game-fireboy-and-watergirl-2',
   'url': 'https://github.com/swanchain/awesome-swanchain/tree/main/game-fireboy-and-watergirl-2'},
  {'name': 'aleo_testnet_miner_beta_f2p

### Estimate Payment amount (optional)

Estimate the cost of deploying the task on specified hardware for duration (in seconds)

In [6]:
duration = 3600 # 1 hour or 3600 seconds

amount = swan_orchestrator.estimate_payment(
    duration=duration, # Optional: Defaults to 3600 seconds or 1 hour
    hardware_id=hardware_id, # Optional: Defaults to 0 (free tier)
)

print(amount)

0


## Auto Pay Path

### Create task (auto_pay)

This step creates, pays and deploys task. It also gets `task_uuid`, which is useful in task info steps.

`wallet_address` is mandatory. `job_source_uri` is mandatory in this demo, please check out example-demo-prebuilt-image if deploying prebuilt images with `image_name`.

In this section, `private_key` is mandatory. If you do not want to use `private_key` and pay through SDK, please check out [create task with no auto_pay](#Create-task-(No-auto_pay))

`job_source_uri` is repository url of code to be deployed, must contain a must contain a dockerfile

In [7]:
# job_source_uri = '<github repository url or lagrange space url of code to be deployed>'

# Demo example: uncomment to use
# job_source_uri = 'https://swanhub-cali.swanchain.io/spaces/0ab9e744-f75b-4bfe-8366-35eac0a8dfa6'

result = swan_orchestrator.create_task(
    wallet_address=wallet_address,
    # job_source_uri=job_source_uri,
    app_repo_image='tetris',
    auto_pay=True, # Optional: Defaults to false, but in this section's path, set to True
    private_key=private_key,
    hardware_id=0, # Optional: Defaults to 0 (free tier)
    region='global', # Optional: Defaults to global
    duration=3600, # Optional: Defaults to 3600 seconds
)

print(json.dumps(result, indent=2))

# Store the task_uuid of created task
task_uuid = result['id']

2024-08-19 15:27:11,774 - INFO - Using C1ae.small machine, hardware_id=0 region='global' duration=3600 (seconds)
2024-08-19 15:27:20,063 - INFO - Payment submitted, task_uuid='4c81e94f-90de-42c8-a4c7-d51ab8f194de', duration=3600, hardware_id=0. Got tx_hash='0x9f95f2dce6e3dbf6d35671839655bb08e2c633183a923ec1e9cae6db426a3f56'
2024-08-19 15:27:23,389 - INFO - Payment validation request sent, task_uuid='4c81e94f-90de-42c8-a4c7-d51ab8f194de', tx_hash='0x9f95f2dce6e3dbf6d35671839655bb08e2c633183a923ec1e9cae6db426a3f56'
2024-08-19 15:27:23,395 - INFO - Payment submitted and validated successfully, task_uuid='4c81e94f-90de-42c8-a4c7-d51ab8f194de', tx_hash='0x9f95f2dce6e3dbf6d35671839655bb08e2c633183a923ec1e9cae6db426a3f56'
2024-08-19 15:27:23,397 - INFO - Task created successfully, task_uuid='4c81e94f-90de-42c8-a4c7-d51ab8f194de', tx_hash='0x9f95f2dce6e3dbf6d35671839655bb08e2c633183a923ec1e9cae6db426a3f56'


{
  "data": {
    "config_id": 1,
    "created_at": 1724095632,
    "duration": 3600,
    "ended_at": null,
    "error_code": null,
    "id": 248,
    "order_type": "Creation",
    "preferred_cp_list": null,
    "refund_tx_hash": null,
    "region": "global",
    "space_id": null,
    "start_in": 300,
    "started_at": 1724095632,
    "status": "pending_payment_confirm",
    "task_uuid": "4c81e94f-90de-42c8-a4c7-d51ab8f194de",
    "tx_hash": "0x9f95f2dce6e3dbf6d35671839655bb08e2c633183a923ec1e9cae6db426a3f56",
    "updated_at": 1724095643,
    "uuid": "9f0c5b6c-ccb1-4224-bd19-4f5d5d7e1ea8"
  },
  "message": "Query order status success.",
  "status": "success",
  "tx_hash": "0x9f95f2dce6e3dbf6d35671839655bb08e2c633183a923ec1e9cae6db426a3f56",
  "id": "4c81e94f-90de-42c8-a4c7-d51ab8f194de",
  "task_uuid": "4c81e94f-90de-42c8-a4c7-d51ab8f194de"
}


In [8]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [],
    "jobs": [],
    "task": {
      "comments": null,
      "created_at": 1724095632,
      "end_at": 1724099232,
      "id": 243,
      "leading_job_id": null,
      "name": null,
      "refund_amount": null,
      "refund_wallet": "0x08ae4dAb3f8B46B9787857E59Cad59af9C650916",
      "source": "v2",
      "start_at": 1724095632,
      "start_in": 300,
      "status": "initialized",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1724095632,
        "dcc_selected_cp_list": null,
        "duration": 3600,
        "end_at": 1724099232,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://swanhub-cali.swanchain.io/spaces/f36176e0-1b33-4da3-b2d2-9d185e46b628",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "preferred_cp_list": null,
          "

### renew task (auto-pay) (optional)

Extend `task_uuid` by `duration`. Using auto pay automatically makes a transaction to SWAN contract and extends the task.

In [9]:
renew_task = swan_orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    auto_pay=True, # Optional: Defaults to False, in this demo path set to True
    private_key=private_key,
    hardware_id=hardware_id # Optional: Defaults to 0 (free tier)
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed task")

2024-08-19 15:27:30,112 - INFO - Payment submitted, task_uuid='4c81e94f-90de-42c8-a4c7-d51ab8f194de', duration=60, hardware_id=0. Got tx_hash='0xde4d009fa2c58b9800cd7936c41f6c01ccbc6e823db73a458a24b84a96059178'
2024-08-19 15:27:30,360 - INFO - Task renewal request sent successfully, task_uuid='4c81e94f-90de-42c8-a4c7-d51ab8f194de' tx_hash='0xde4d009fa2c58b9800cd7936c41f6c01ccbc6e823db73a458a24b84a96059178', duration=60


successfully renewed task


If completed steps above, please go to [View Deployed Task Results](#View-Deployed-Task-Results) to view results, or [terminate task](#terminate-task) for more functions related to tasks

## No Auto Pay Path

### Create task (No auto_pay)

This creates task, but does not pay or deploy task. It gets `task_uuid`, which is useful in payment steps.

`wallet_address` is mandatory. `job_source_uri` is mandatory in this demo, please check out example-demo-prebuilt-image if deploying prebuilt images.

`job_source_uri` is repository url of code to be deployed, must contain a must contain a dockerfile

In [10]:
job_source_uri = '<github repository url or lagrange space url of code to be deployed>'

# Demo example: uncomment to use
# job_source_uri = 'https://github.com/alphaflows/tetris-docker-image.git'

result = swan_orchestrator.create_task(
    wallet_address=wallet_address,
    job_source_uri=job_source_uri,
    hardware_id=hardware_id, # Optional: Defaults to 0 (free tier)
    region='global', # Optional: Defaults to global
    duration=duration, # Optional: Defaults to 3600 seconds
    auto_pay=False, # Optional: Defaults to false
)

print(json.dumps(result, indent=2))

# Store the task_uuid of created task
task_uuid = result['id']

2024-08-19 15:27:30,550 - INFO - Using C1ae.small machine, hardware_id=0 region='global' duration=3600 (seconds)
2024-08-19 15:27:30,895 - INFO - Task created successfully, task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4', tx_hash=None


{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1724095650,
      "end_at": 1724099250,
      "id": 244,
      "leading_job_id": null,
      "name": null,
      "refund_amount": null,
      "refund_wallet": "0x08ae4dAb3f8B46B9787857E59Cad59af9C650916",
      "source": "v2",
      "start_at": 1724095650,
      "start_in": 300,
      "status": "initialized",
      "task_detail": {
        "amount": null,
        "bidder_limit": 3,
        "created_at": 1724095650,
        "dcc_selected_cp_list": null,
        "duration": 3600,
        "end_at": 1724099250,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "<github repository url or lagrange space url of code to be deployed>",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "preferred_cp_list": null,
          "region": "global",
          "storage": null,
         

### Make Payment (Optional)

This step is using `task_uuid`, `private_key`,  `duration`, and `hardware_id` to submit payment and make task eligible for assigning if payment successful via swan SDK. `task_uuid`, `private_key` are mandatory.

If following this section, please skip to - [View Deployed Task Results](#View-Deployed-Task-Results) next.
If do not want to submit payment with swan SDK, and directly made payment to swan contract instead, please move onto next section.

In [11]:
if result_validation := swan_orchestrator.make_payment(
    task_uuid=task_uuid, 
    private_key=private_key,
    duration=3600, # Optional: Defaults to 3600 seconds (1 hour)
    hardware_id=0 # Optional: Defaults to 0 (free tier)
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

2024-08-19 15:27:38,097 - INFO - Payment submitted, task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4', duration=3600, hardware_id=0. Got tx_hash='0xa849f7467c62b21103f9647d5d512ee76b1b6e94efdba9bd55e80bedbdb5f445'
2024-08-19 15:27:41,347 - INFO - Payment validation request sent, task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4', tx_hash='0xa849f7467c62b21103f9647d5d512ee76b1b6e94efdba9bd55e80bedbdb5f445'
2024-08-19 15:27:41,347 - INFO - Payment submitted and validated successfully, task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4', tx_hash='0xa849f7467c62b21103f9647d5d512ee76b1b6e94efdba9bd55e80bedbdb5f445'


{
  "data": {
    "config_id": 1,
    "created_at": 1724095650,
    "duration": 3600,
    "ended_at": null,
    "error_code": null,
    "id": 250,
    "order_type": "Creation",
    "preferred_cp_list": null,
    "refund_tx_hash": null,
    "region": "global",
    "space_id": null,
    "start_in": 300,
    "started_at": 1724095650,
    "status": "pending_payment_confirm",
    "task_uuid": "6571cb3b-9964-49bb-a1d5-29465430eef4",
    "tx_hash": "0xa849f7467c62b21103f9647d5d512ee76b1b6e94efdba9bd55e80bedbdb5f445",
    "updated_at": 1724095661,
    "uuid": "3ee228d4-6d5a-4df5-a4dc-8aac58d12f18"
  },
  "message": "Query order status success.",
  "status": "success",
  "tx_hash": "0xa849f7467c62b21103f9647d5d512ee76b1b6e94efdba9bd55e80bedbdb5f445"
}


### Validate Payment to deploy task

Only use this if paid directly to contract. If used make_payment section above, ignore this section.

This step will use `tx_hash` and `task_uuid` to validate the payment and then make task eligible for assigning if validation successful

Will Delete the submit_payment code block below in final version

In [12]:
if result_validation := swan_orchestrator.validate_payment(
    tx_hash="<tx_hash of payment of task_uuid to swan contract>",
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

2024-08-19 15:27:41,533 - INFO - Payment validation request sent, task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4', tx_hash='<tx_hash of payment of task_uuid to swan contract>'


{
  "data": null,
  "message": "Validation error: when sending a str, it must be a hex string. Got: '<tx_hash of payment of task_uuid to swan contract>'",
  "status": "failed"
}


### Renew Task (no auto-pay) (Optional)

Extend `task_uuid` by `duration`. `tx_hash` of payment for extension required.

In [13]:
renew_task = swan_orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    tx_hash="<tx_hash of payment of task_uuid to swan contract>", # tx_hash of payment to swan contract for this task
    hardware_id=hardware_id # Optional: Defaults to 0 (free tier)
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed {task_uuid}")
else:
    print(f"Unable to renew {task_uuid}")

2024-08-19 15:27:41,537 - INFO - Using given payment transaction hash, tx_hash='<tx_hash of payment of task_uuid to swan contract>'
2024-08-19 15:27:41,708 - INFO - Task renewal request sent successfully, task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4' tx_hash='<tx_hash of payment of task_uuid to swan contract>', duration=60


Unable to renew 6571cb3b-9964-49bb-a1d5-29465430eef4


### terminate task (Optional)

Terminate the task `task_uuid` and get a refund for remaining time

In [14]:
terminate_status = swan_orchestrator.terminate_task(task_uuid)
if terminate_status['status'] == 'success':
    print(f"Terminated {task_uuid} successfully")
else:
    print(f"Failed to terminate {task_uuid}")

Terminated 6571cb3b-9964-49bb-a1d5-29465430eef4 successfully


### claim review (Optional)

Review if `task_uui` uptime is above 90% and give refund if below 90%

In [15]:
claim_review = swan_orchestrator.claim_review(task_uuid)
print(claim_review)

{'data': {'error_code': 1012}, 'message': "task_uuid='6571cb3b-9964-49bb-a1d5-29465430eef4' status (terminated) not `finished`, cannot claim review", 'status': 'failed'}


## View Deployed Task Results

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [16]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [],
    "jobs": [],
    "task": {
      "comments": null,
      "created_at": 1724095650,
      "end_at": 1724095661,
      "id": 244,
      "leading_job_id": null,
      "name": null,
      "refund_amount": null,
      "refund_wallet": "0x08ae4dAb3f8B46B9787857E59Cad59af9C650916",
      "source": "v2",
      "start_at": 1724095650,
      "start_in": 300,
      "status": "terminated",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1724095650,
        "dcc_selected_cp_list": null,
        "duration": 3600,
        "end_at": 1724099250,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "<github repository url or lagrange space url of code to be deployed>",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "preferred_cp_list": null,
          "region": "

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [17]:
result_url = swan_orchestrator.get_real_url(task_uuid)
print(result_url)

[]


Check the response codes of the result_url

In [18]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

for url in result_url:
    response = requests.get(url, headers=headers)

    try:
        print(json.dumps(response.json(), indent=4))
    except Exception as e:
        print(e)
        print(response)